In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper, provideData
from Utils.transform import cutOut

Num GPUs: 1
Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
dimension = (64,64)
channels = 7
optimizer = Adam( lr = 1e-3 )
#slices = [224,352,224,352]
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]

tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true[:,:])
                           
def mixd(output):
    rate = tf.math.exp(output[0,:,:,0]) #A 
    s = tf.math.sigmoid(output[0,:,:,1])
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), #E
     tfd.Poisson(rate=rate) #F 
     ]
    mixture = tfd.Mixture(
          cat=tfd.Categorical(probs=tf.stack([1-s, s],axis=-1)),#D
          components=components)
    
    return mixture
    
def testnetPoisson(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):


    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64

    
    output = Conv2D(16, (1, 1), activation="relu")(up01)  # 1 x 64x64
    output = Flatten()(output)
    
    output = Dense(16*16)(output)
    #output = Activation(tf.math.softplus)(output)
    output = Dropout(0.25)(output)
    output = Dense(64*64)(output)
    output = tfp.layers.IndependentPoisson((64,64,1))(output)
    model = Model(inputs=inputs, outputs=output)
    return model

DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

In [3]:
batch_size=100
tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=None,
                          channels = channels,
                          timeToPred = 5,
                          year = [2015,2016,2017],
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

TypeError: provideData() got an unexpected keyword argument 'years'

In [ ]:
t = Trainer(testnetPoisson,
                    lossfunction=NLL,
                    pathToData=(train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

In [ ]:
t.fit(10)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
history = t.history
def plotHistory():
    # Plot training & validation accuracy values
    plt.plot(history['mse'])
    plt.plot(history['val_mse'])
    plt.title('Model MSE')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plotHistory()

In [ ]:
model = t.model
pred = None
label = None
pred = 10
test.on_epoch_end()
mean = []
stdev = []
label = []
for x,y in test:
    for i in range(batch_size):
        if y[i,:,:,:].max() > 0:
            pred = model(np.array([x[i,:,:,:]]))
            mean.append(pred.mean())
            stdev.append(pred.stddev())
            label.append(y[i,:,:,:])
        #mean = model(x).mean()
        #std = model(x).stddev()
        #label = y
        break

In [ ]:
fig, axes = plt.subplots(10, 3, figsize=(16,8*10),dpi=64)
#fig.set_title(["mean","stdev","label"])
for batch,img in enumerate(mean):
    if batch == 10:
        break
    axes[batch,0].imshow(img[0,:,:,0],cmap="gray")
    axes[batch,1].imshow(stdev[batch][0,:,:,0],cmap="gray")
    axes[batch,2].imshow(label[batch][0,:,:],cmap="gray")

In [ ]:
import numpy as np
class ToUint8(object):
    """docstring for ToUint8"""
    def __init__(self):
        super(ToUint8, self).__init__()
              
        
    def __call__(self,img):
        return (img[0,:,:,0].mean() * 255).astype(np.uint8)

In [ ]:

from Evaluation import Evaluation
from Utils.Data import provideData
model = t.model
BATCH_SIZE = 1
CHANNELS = 7
NUMBER_OF_TIMESTEPS = 2000
NUMBER_OF_PREDICTIONS = 1
evaluation = Evaluation('Poisson_480x480_30min_Eval',
                        Data=provideData(batch_size=BATCH_SIZE, dimension=dimension,
                                         preTransformation=PRETRAINING_TRANSFORMATIONS, 
                                         transform = [], 
                                         channels=CHANNELS,
                                         timeToPred = 30),
                        batch_size=BATCH_SIZE,
                        channels=CHANNELS,
                        dimension=dimension,
                        number_of_predictions=NUMBER_OF_PREDICTIONS,
                        time_steps=NUMBER_OF_TIMESTEPS,
                        transform_predictions=[ToUint8()],
                        model=model
                        
                        )
!rm -rf ./Evaluation
evaluation.make_predictions()

In [ ]:
evaluation.create_grayscale_gif(200)
evaluation.evaluation(rain_no_rain=True, four_categories=True, weather_changes_two_categories=True)
evaluation.create_report()